In [2]:
from automation_script import*

In [ ]:
from automation_script import *

automation = AutomationProcess()
automation.add_input_path('inputs/UpworkSampleCSV.csv')\
            .add_output_path('outputs/')\
            .automate()

In [4]:
!pip install xlsxwriter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 7.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
automation = AutomationProcess()
automation.add_input_path('inputs/UpworkSampleCSV.csv')\
            .add_output_path('outputs/')\
            .automate()

'Pipeline Finished Processing'

In [3]:
files_addresses = os.listdir('outputs/')

def df_generator(files):
    for i in files:
        yield pd.read_csv(f'outputs/{i}')

In [4]:
df1 = next(df_generator(files_addresses))
df2 = next(df_generator(files_addresses))

In [5]:
df1

,listing_id,business_name,address,orgid,Rating,Rating Count,phone,website,domain,chain_name,...,lon,in_chain,in_search,Category1,Category2,Category3,Category4,Category5,Category6,Category7
0,CID:5097641582142782286,PhotoWorks Interactive Photobooth Rentals of S...,"1550 dell ave, campbell, ca 95008",photoworksinteractive,5.0,7,(800) 990-8445,photoworksinteractive.com,photoworksinteractive,Photoworks Interactive,...,-121.955669,True,True,Photography Service,Party Supply,Printing & Copy Shop,NaN,NaN,NaN,NaN


In [6]:
df2

,listing_id,business_name,address,orgid,Rating,Rating Count,phone,website,domain,chain_name,...,lon,in_chain,in_search,Category1,Category2,Category3,Category4,Category5,Category6,Category7
0,CID:5097641582142782286,PhotoWorks Interactive Photobooth Rentals of S...,"1550 dell ave, campbell, ca 95008",photoworksinteractive,5.0,7,(800) 990-8445,photoworksinteractive.com,photoworksinteractive,Photoworks Interactive,...,-121.955669,True,True,Photography Service,Party Supply,Printing & Copy Shop,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

# Create the Excel writer object
writer = pd.ExcelWriter('combined_dataframes.xlsx', engine='xlsxwriter')

# Define a function to write a dataframe to the Excel file with its name as a header
def write_dataframe(df, sheet_name):
    df.to_excel(writer, sheet_name=sheet_name, startrow=1, header=False)
    worksheet = writer.sheets[sheet_name]
    worksheet.write_string(0, 0, sheet_name)

# Loop through your dataframes and write them to the Excel file
for i, df in enumerate([df1, df2, df3, df4, df5, df6, df7, df8]):
    write_dataframe(df, 'Dataframe {}'.format(i+1))

# Save the Excel file
writer.save()


In [5]:
import pandas as pd
import os

# Define a class to hold automation related data and methods
class Automation:
    
    def __init__(self):
        self.input_path = None
        self.output_path = None
        self.data = None
        self.unique_addresses = None

    # Set the input file path
    def set_input_path(self, input_path):
        self.input_path = input_path

    # Set the output file path
    def set_output_path(self, output_path):
        self.output_path = output_path

    # Load data into the object
    def load_data(self, data):
        self.data = data

    # Set unique addresses in the object
    def set_unique_addresses(self, addresses):
        self.unique_addresses = addresses


# Define a class to hold automation process related methods
class AutomationProcess:
    
    # Instantiate Automation
    def __init__(self):
        self.automation = Automation()
        
    # Set the input file path in the Automation object
    def add_input_path(self, input_path):
        self.automation.set_input_path(input_path)
        return self

    # Set the output file path in the Automation object
    def add_output_path(self, output_path):
        self.automation.set_output_path(output_path)
        return self

    # Load data into the Automation object
    def load_dataframe_from_csv(self):
        # Check if the input file path exists and is a valid file
        if not self.automation.input_path or not os.path.isfile(self.automation.input_path):
            raise ValueError("Input path is not set or is not a valid file.")
        # Load the csv file into the object
        self.automation.load_data(pd.read_csv(self.automation.input_path))

    # Standardize the address column in the data
    def standardize_address(self):
        # Convert the address column to lowercase, strip whitespace, and remove extra spaces
        self.automation.data['address'] = self.automation.data['address'].str.lower().str.strip().str.replace('\s+', ' ', regex=True)

    # Set unique addresses in the Automation object
    def get_unique_addresses(self):
        self.automation.set_unique_addresses(self.automation.data['address'].unique())

    # Sort a dataframe by the rating column
    def sort_by_ratings(self, df):
        return df.sort_values(by='Rating', ascending=False)
    
    # Save the data in a csv file
    def save_data(self, df, address):
        filename = f"{self.automation.output_path + address}.csv"
        df.to_csv(filename, index=False)
        
    # Define a function to write a dataframe to the Csv file with its name as a header
    def write_dataframe(self, df, address):
        # open a file object in append mode
        with open('output.csv', 'a') as f:
            # write the first dataframe to the file
            if f.tell() != 0:
                f.write('\n')
            f.write(f'{address}\n')
            df.to_csv(f, index=False)


    
    # Iterate through unique addresses and update categories
    def iterate_addresses(self):
        df = self.automation.data 
        for address in self.automation.unique_addresses:
            # Sort the dataframe by rating for a specific address
            df_filtered = df.loc[df['address'] == address]
            if df_filtered.shape == (1, len(df.columns)):
                # If there is only one row, directly save it to CSV and continue with the next address
                self.save_data(df_filtered, address)
                self.write_dataframe(df_filtered, address)
                continue

            # Sort the dataframe by rating for a specific address
            df_sorted = self.sort_by_ratings(df_filtered)
            
            # Set the in_search column to False for all but the first row
            df_sorted.iloc[1:, df_sorted.columns.get_loc('in_search')] = False
            
            # Reset the index to start from 0
            df_sorted = df_sorted.reset_index(drop=True)
            
            # Get the category values for all but the first row
            values = df_sorted['Category1'][1:].tolist()
            
            # Get all the category column names
            columns = [col for col in df_sorted.columns if col.startswith('Category')]
            
            # Update category columns with values from subsequent rows
            for col in columns:
                if not values:
                    break
                if pd.isna(df_sorted.loc[0, col]):
                    df_sorted.loc[0, col] = values.pop(0)
            
            # Save the updated data to a CSV file
            self.save_data(df_sorted, address)
            self.write_dataframe(df_sorted, address)
        
        return 'Pipeline Finished Processing'

    # Runs the Automation process
    def automate(self):
        self.load_dataframe_from_csv()
        self.standardize_address()
        self.get_unique_addresses()
        self.iterate_addresses()
        return 'Pipeline Finished Processing'


In [8]:
import pandas as pd

# create some example dataframes
df1 = pd.DataFrame({'Name': ['John', 'Jane', 'Jack', 'Jill'], 'Age': [35, 27, 42, 29]})
df2 = pd.DataFrame({'Country': ['USA', 'China', 'Japan', 'Germany'], 'Population': [328.2, 1400, 126.3, 83.2], 'GDP': [21.4, 14.1, 5.2, 4.2]})

# open a file object in append mode
with open('output.csv', 'a') as f:
    # write the first dataframe to the file
    f.write('Dataframe 1\n')
    df1.to_csv(f, index=False)
    
    # write the second dataframe to the file
    f.write('Dataframe 2\n')
    df2.to_csv(f, index=False)


In [9]:
import pandas as pd

# read the data from the file into a list of dataframes
df_list = []
with open('output.csv', 'r') as f:
    while True:
        # read the label for the next dataframe
        label = f.readline().strip()
        if not label:
            break  # end of file
        
        # read the dataframe from the file
        df = pd.read_csv(f)
        
        # add the dataframe to the list with the given label
        df_list.append((label, df))

# print the contents of the dataframes
for label, df in df_list:
    print(f"{label}:\n{df}\n")


ParserError: Error tokenizing data. C error: Expected 2 fields in line 7, saw 3
